## Data Cleaning with Python and Pandas
Practical Block 4

### Part 1: Using read_csv()

In [29]:
import pandas as pd
import numpy as np
pd.read_csv("C:/Users/ondro/Data Science Masters/data/unnamed.txt")

,21,6.0
0,42,5.9
1,18,5.7*
2,21,NA


In [4]:
# Read again with header = None
pd.read_csv("C:/Users/ondro/Data Science Masters/data/unnamed.txt", header = None)

,0,1
0,21,6.0
1,42,5.9
2,18,5.7*
3,21,NA


In [5]:
# Set the column names
pd.read_csv("C:/Users/ondro/Data Science Masters/data/unnamed.txt", header = None, names = ("age", "height"))

,age,height
0,21,6.0
1,42,5.9
2,18,5.7*
3,21,NA


In [7]:
# Read this into a Python object
person = pd.read_csv("C:/Users/ondro/Data Science Masters/data/unnamed.txt", header = None, names = ("age", "height"))
person

,age,height
0,21,6.0
1,42,5.9
2,18,5.7*
3,21,NA


In [8]:
# Examine with .info()
person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   age     4 non-null      int64 
 1   height  4 non-null      object
dtypes: int64(1), object(1)
memory usage: 196.0+ bytes


In [9]:
# try to import the data with height as numeric
person = pd.read_csv("C:/Users/ondro/Data Science Masters/data/unnamed.txt", 
                    header = None, 
                    names = ("age", "height"),
                    dtype = {"age":"int64", "height":"float64"})

ValueError: could not convert string to float: '5.7*'

ValueError: could not convert string to float: '5.7*'

In [19]:
# So, read height as an object and then coerce it
person['height'] = pd.to_numeric(person['height'], errors = 'coerce')
person

,age,height
0,21,6.0
1,42,5.9
2,18,NaN
3,21,NaN


### Part 2: Dealing with unstructured text data

Step 1: readlines()

In [21]:
myfile = open("C:/Users/ondro/Data Science Masters/data/daltons.txt")
lines = myfile.readlines()
lines

['%% Data on the Dalton Brothers\n',
 'Gratt,1861,1892\n',
 'Bob,1892\n',
 '1871,Emmet,1937\n',
 '% Names, birth and death dates']

Step 2: select lines only with data using regular expressions

In [37]:
# import re library for regular expressions
import re

# remove comments starting with %
dat = [line for line in lines if not (line.startswith('%'))]
dat

['Gratt,1861,1892\n', 'Bob,1892\n', '1871,Emmet,1937\n']

Step 3: Split lines into fields

In [23]:
# create a list using .split
fieldList = [line.split(",") for line in dat]
fieldList

[['Gratt', '1861', '1892\n'], ['Bob', '1892\n'], ['1871', 'Emmet', '1937\n']]

Step 4: Standardise rows

In [49]:
def assignFields(e):
    # create a list to hold the extracted fields
    out = ['','','']
    
    # extract the name value to the first position
    out[0]=list(filter(lambda x:re.search(x),e))[0]
    
    # extract the birth date (if any) 
    # all were born before 1890 and died after 1890
    # to_numeric()
    # if it cannot coerce to numeric will return an empty list
    # which is then replaced by NaN
    
    i = list(filter(lambda x:pd.to_numeric(x, errors = 'coerce')<1890, e))
    out[1] = pd.to_numeric(i[0]) if len(i)> 0 else np.nan
    
    i = list(filter(lambda x:pd.to_numeric(x, errors = 'coerce')>1890, e))
    out[2] = pd.to_numeric(i[0]) if len(i)> 0 else np.nan
    
    return(out)
             

In [50]:
# map the function over fieldList
standardFields = list(map(assignFields, fieldList))
standardFields

TypeError: expected string or bytes-like object, got 'int'

Step 5: Transform to a data frame

In [32]:
pd.DataFrame(standardFields, columns = ['name', 'birth', 'death'])

,name,birth,death
0,,1892,
1,,1892,
2,,1937,
